In [38]:

import os, types
from ibm_watson_machine_learning import APIClient
import pandas as pd
from botocore.client import Config
import ibm_boto3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense,BatchNormalization
def __iter__(self): return 0

bucket = 'imagepreprocessing-donotdelete-pr-scxfs2qs5hupzk'
object_key = 'FingerDataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [39]:
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
filepaths=unzip.namelist()
for path in filepaths:
    unzip.extract(path)

In [40]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                    rotation_range=10.,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    zoom_range=0.2,
                                    horizontal_flip=True)

In [41]:
train_gen = train_datagen.flow_from_directory(
        r'/home/wsuser/work/FingerDataset/train',
        target_size=(128,128),
        color_mode='grayscale',
        batch_size=32,
        classes=['0','1','2','3','4','5'],
        class_mode='categorical'
    )

Found 594 images belonging to 6 classes.


In [42]:
test_datagen  = ImageDataGenerator( rescale=1./255 )

In [43]:
test_gen = test_datagen.flow_from_directory(
        r'/home/wsuser/work/FingerDataset/test',
        target_size=(128,128),
        color_mode='grayscale',
        batch_size=32,
        classes=['0','1','2','3','4','5'],
        class_mode='categorical'
    )

Found 30 images belonging to 6 classes.


In [44]:
model=Sequential()

In [46]:
model.add(BatchNormalization(input_shape = (128,128,1)))
model.add(Convolution2D(32, (3,3), activation ='relu', input_shape = (128, 128, 1))) 
model.add(MaxPooling2D(pool_size=2))
model.add(Convolution2D(filters=6,kernel_size=4,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Convolution2D(filters=128,kernel_size=3,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Convolution2D(filters=128,kernel_size=2,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten()) 
model.add(Dense(units=128,activation = 'relu'))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dense(units = 6, activation = 'softmax'))

In [47]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 128, 128, 1)      4         
 ormalization)                                                   
                                                                 
 batch_normalization_1 (Batc  (None, 128, 128, 1)      4         
 hNormalization)                                                 
                                                                 
 conv2d (Conv2D)             (None, 126, 126, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 63, 63, 6)         3078      
                                                      

In [50]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])
model.fit(train_gen,
                    steps_per_epoch=len(train_gen),
                    epochs=10,
                    validation_data=test_gen,
                    validation_steps=len(test_gen))

Epoch 1/10
19/19 [==============================] - 11s 523ms/step - loss: 1.2543 - accuracy: 0.4646 - val_loss: 0.9312 - val_accuracy: 0.8667
Epoch 2/10
19/19 [==============================] - 10s 512ms/step - loss: 0.9057 - accuracy: 0.6380 - val_loss: 0.6313 - val_accuracy: 0.8667
Epoch 3/10
19/19 [==============================] - 10s 505ms/step - loss: 0.6441 - accuracy: 0.7323 - val_loss: 0.4718 - val_accuracy: 0.8667
Epoch 4/10
19/19 [==============================] - 10s 507ms/step - loss: 0.4827 - accuracy: 0.8266 - val_loss: 0.3501 - val_accuracy: 0.9667
Epoch 5/10
19/19 [==============================] - 10s 510ms/step - loss: 0.3545 - accuracy: 0.8519 - val_loss: 0.3494 - val_accuracy: 0.9333
Epoch 6/10
19/19 [==============================] - 10s 509ms/step - loss: 0.2757 - accuracy: 0.8906 - val_loss: 0.2628 - val_accuracy: 0.9667
Epoch 7/10
19/19 [==============================] - 10s 505ms/step - loss: 0.3496 - accuracy: 0.8687 - val_loss: 0.2235 - val_accuracy: 0.9667

In [51]:
model.save('gesture.h5')

In [52]:
!tar -zcvf gesture-classification.tgz gesture.h5

gesture.h5


In [53]:
!pip install watson-machine-learning-client

     |████████████████████████████████| 538 kB 21.0 MB/s eta 0:00:01


In [54]:
!pip install ibm_watson_machine_learning

In [56]:
wml_credentials={
    "url":'https://us-south.ml.cloud.ibm.com',
    "apikey":'c-CNFpHvbBnkyycETjYhP3gj55rflfWVJHQzkALSe3iZ'
}
client = APIClient(wml_credentials)
client

In [58]:
client.spaces.get_details()

{'resources': [{'entity': {'compute': [{'crn': 'crn:v1:bluemix:public:pm-20:us-south:a/0089df65ef5844529d3c2c6c003715f1:056d408a-9c6f-4b53-97b2-2cc51faca197::',
      'guid': '056d408a-9c6f-4b53-97b2-2cc51faca197',
      'name': 'Watson Machine Learning-si',
      'type': 'machine_learning'}],
    'description': 'Deployment space for Image Processing',
    'name': 'imageprocessing',
    'scope': {'bss_account_id': '0089df65ef5844529d3c2c6c003715f1'},
    'stage': {'production': False},
    'status': {'state': 'active'},
    'storage': {'properties': {'bucket_name': 'f7fdd78e-d313-429a-8af1-626b6b35a52b',
      'bucket_region': 'us-south',
      'credentials': {'admin': {'access_key_id': 'c9ae9503f4174947b85e72f03c7e751f',
        'api_key': '4Wv5Y9bZjV_nezJ7CIh3GDlHvKIqViRQNMKQ_0WE7z6s',
        'secret_access_key': '2e15e28af72cc58dd867b50992c87873c3d3644ec06b2bed',
        'service_id': 'ServiceId-14763884-f649-4666-88fc-d2333a2c8ab4'},
       'editor': {'access_key_id': 'f91844bdc7a

In [59]:
def guid_space_name(client,space_name):
  space = client.spaces.get_details()
  return(next(item for item in space['resources'] if item['entity']['name']==space_name)['metadata']['id'])

In [60]:
space_uid = guid_space_name(client,'imageprocessing')
space_uid

'1a0495a2-b261-4802-8f30-51ac689a0c56'

In [61]:
client.set.default_space(space_uid)

'SUCCESS'

In [62]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [63]:
software_space_uid = client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [64]:
model_details = client.repository.store_model(model='gesture-classification.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN B2 Model",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid
})

In [65]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'software_spec': {'id': 'acd9c798-6974-5d2f-a657-ce06e986df4d',
   'name': 'tensorflow_rt22.1-py3.9'},
  'type': 'tensorflow_2.7'},
 'metadata': {'created_at': '2022-11-12T11:48:26.692Z',
  'id': 'deba26cb-a718-4b61-b764-5db71ebb5cb6',
  'modified_at': '2022-11-12T11:48:30.718Z',
  'name': 'CNN B2 Model',
  'owner': 'IBMid-665002MNB6',
  'resource_key': '079a89e6-fe6a-4341-8037-f28ae62aa48e',
  'space_id': '1a0495a2-b261-4802-8f30-51ac689a0c56'},
 'system': {'warnings': []}}

In [66]:
model_id = client.repository.get_model_id(model_details)
model_id

'deba26cb-a718-4b61-b764-5db71ebb5cb6'

In [68]:
client.repository.download(model_id,'B2_IBM_model.tar.gb')

Successfully saved model content to file: 'B2_IBM_model.tar.gb'


'/home/wsuser/work/B2_IBM_model.tar.gb'